In [1]:
library(tidyverse)
library(dplyr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


**Individual Project Planning**

Ann Huynh

***1) Data Description***

Players Dataset (players.csv)
- 7 columns, 196 rows, 1372 variables
- columns:
  - experience (categorical: Pro, Veteran, Regular, Amateur, Beginner) -> the playing experience of the player
  - subscribe (factor: True/False) -> whether the player subscribed to the game newsletter
  - hashedEmail (character) -> the hashed email of the player
  - played_hours (numeric) -> the number of hours played by the player
  - name (character) -> the name of the player
  - gender (categorical: Female, Male) -> the gender of the player
  - Age (numeric) -> the age of the player

Sessions Dataset (sessions.csv)
- 5 columns, 1535 rows, 7675 variables
- columns:
  - hashedEmail (character) -> hashed email of the player
  - start_time (numeric) -> start date and time. date in day/month/year and time in military time
  - end_time (numeric) -> end date and time. date in day/month/year and time in military time
  - original_start_time -> start time before converting to start_time
  - original_end_time -> end time before converting to end_time

Potential Issues with data:

- In the sessions dataset, the start_time and end_time columns contain more than one value. The columns have
  both date and time. This may need to be modified so that date and time are separate.
- original_start_time and original_end_time are not easily interpretable numbers so they must be converted or
  different columns should be used (like start_time or end_time)


Summary Statistics: 
For players dataset, 
- The average number of hours played is 5.85 hours
- The average age is 21.14 years

The code below computes these values. 

  
  

In [2]:
players <- read_csv("individual_planning_project/players.csv")


ERROR: Error: 'individual_planning_project/players.csv' does not exist in current working directory ('/home/jovyan/work/individual_planning_project').


In [3]:
avg_hours <- players |> 
    summarise(avg_hours = mean(played_hours))

avg_hours

ERROR: Error in eval(expr, envir, enclos): object 'players' not found


In [4]:
avg_age <- players |> 
    summarise(avg_age = mean(Age, na.rm = TRUE))
    

avg_age

ERROR: Error in eval(expr, envir, enclos): object 'players' not found


***2) Questions***

- Broad Question: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
- Specific Question: Does the experience of a player predict whether or not they will subscribe to a game-related newsletter in the players dataset?

The player's dataset contains the experience (experience column) of each player and whether or not a player has already subscribed to a game-related newsletter. These variables will be important when addressing the specific question. 
Data wrangling may be necessary as the experience column is categorical, meaning that grouping might be needed in order to extract information. 

***3) Exploratory Data Analysis and Visualization***

***4) Methods and Plans***